In [3]:
!pip install --quiet ragas

In [3]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness, context_recall, answer_correctness, context_precision
import os, openai

In [4]:
with open('../sample_data/token.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [6]:
def ragas_evaluate(eval_sets):
    return evaluate(
    eval_sets,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness
    ]
  )

In [7]:
busan_context_right = [
  "Busan is divided into 15 major administrative districts and a single county, together housing a population of approximately 3.6 million."
  "Busan (Korean: 부산, pronounced [pusan]), officially Busan Metropolitan City, is South Korea's second most populous city after Seoul, with a population of over 3.4 million inhabitants as of 2017.[4] Formerly romanized as Pusan (and Fuzan under Japanese rule)."
  "부산광역시(釜山廣域市, 영어: Busan Metropolitan City)는 한반도 남동부에 위치한 광역시이다. 대한민국의 제2의 도시이자 최대의 해양(항구) 도시이며, 부산항을 중심으로 해상 무역과 물류 산업이 발달하였다. 일본과는 대한해협을 사이에 두고 마주하고 있다. 시청 소재지는 연제구 연산동이며, 행정구역은 15구 1군이다."
]

In [8]:
busan_context_wrong_1 = ["부산에는 호떡이 맛있다.", "부산에는 언덕이 많다."]
busan_context_wrong_2 = ["호떡이 맛있다.", "언덕이 많다."]

# Correct Answer & Correct Context

In [16]:
# 정답, 그리고 좋은 컨텍스트
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],     # 질문
	"answer"  : ["대한민국에서 두번째로 큰 도시는 부산입니다."],  #  언어 모델이 내놓은 답
	"contexts": [busan_context_right],                         # 참고 자료
	"ground_truth": ["부산"]                                   # 정답
}

In [17]:
eval_sets = Dataset.from_dict(eval_set)

In [18]:
eval_sets

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 1
})

In [19]:
# 정답 세트!
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9950, 'context_recall': 1.0000, 'answer_correctness': 0.9729}

# Wrong Answer & Correct Context

In [20]:
# 오답, 그러나 좋은 컨텍스트. 답을 줬는데 왜 대답을 제대로 못해...
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],
	"answer"  : ["부산은 한국에 있다."],
	"contexts": [busan_context_right],
	"ground_truth": ["부산"]
}

In [21]:
eval_sets = Dataset.from_dict(eval_set)

In [22]:
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.8117, 'context_recall': 1.0000, 'answer_correctness': 0.7295}

# Correct Answer & Wrong Context

In [23]:
# 정답, 그러나 불량 컨텍스트. 정답을 어떻게 한 거니??
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],
	"answer"  : ["대한민국에서 두번째로 큰 도시는 부산"],
	"contexts": [busan_context_wrong_1],
	"ground_truth": ["부산"]
}

In [24]:
eval_sets = Dataset.from_dict(eval_set)

In [25]:
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 0.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9950, 'context_recall': 1.0000, 'answer_correctness': 0.9745}

In [26]:
# 정답, 그러나 불량 컨텍스트. 정답을 어떻게 한 거니?? 컨텍스트에는 부산이란 단어도 없어!!
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],
	"answer"  : ["대한민국에서 두번째로 큰 도시는 부산"],
	"contexts": [busan_context_wrong_2],
	"ground_truth": ["부산"]
}

In [27]:
eval_sets = Dataset.from_dict(eval_set)

In [28]:
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.9950, 'context_recall': 0.0000, 'answer_correctness': 0.9745}

# Wrong Answer & Correct Context

In [29]:
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],
	"answer"  : ["대한민국에서 두번째로 큰 도시는 대구입니다."],
	"contexts": [busan_context_right],
	"ground_truth": ["부산"]
}

In [30]:
eval_sets = Dataset.from_dict(eval_set)

In [31]:
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 0.9950, 'context_recall': 1.0000, 'answer_correctness': 0.2067}

In [32]:
# 오답, 그러나 좋은 컨텍스트.
eval_set = {
	"question": ["대한민국에서 두번째로 큰 도시는 어디야?"],
	"answer"  : ["대한민국에서 두번째로 큰 도시는 대구입니다."],
	"contexts": [busan_context_right],
	"ground_truth": ["부산은 호떡이 맛있다"]
}

In [33]:
eval_sets = Dataset.from_dict(eval_set)

In [34]:
ragas_evaluate(eval_sets)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.9950, 'context_recall': 0.0000, 'answer_correctness': 0.2018}